# Pandas Avanzado para Data Science

Este notebook cubre técnicas avanzadas de pandas esenciales para proyectos profesionales de data science. Los contenidos incluyen análisis temporal, optimización de performance, manejo avanzado de datos y técnicas especializadas.

## Contenido
1. [Series Temporales y Manejo de Fechas](#Series-Temporales-y-Manejo-de-Fechas)
2. [Análisis de Correlaciones](#Análisis-de-Correlaciones)
3. [Tratamiento Avanzado de Valores Faltantes](#Tratamiento-Avanzado-de-Valores-Faltantes)
4. [Pivot Tables Complejas](#Pivot-Tables-Complejas)
5. [Manejo de Texto y Datos Categóricos](#Manejo-de-Texto-y-Datos-Categóricos)
6. [Performance y Optimización](#Performance-y-Optimización)
7. [Joins y Merges Avanzados](#Joins-y-Merges-Avanzados)
8. [Detección y Tratamiento de Outliers](#Detección-y-Tratamiento-de-Outliers)
9. [Visualización con Pandas](#Visualización-con-Pandas)
10. [MultiIndex y Estructuras Complejas](#MultiIndex-y-Estructuras-Complejas)

# Series Temporales y Manejo de Fechas

![Series Temporales y Manejo de Fechas](imgs/series_temporales.png)


Las series temporales son fundamentales en data science para analizar datos que cambian con el tiempo. Pandas proporciona herramientas potentes para trabajar con fechas, índices temporales y análisis de tendencias.

## Conversión a Tipos de Fecha

La conversión correcta de fechas es fundamental para el análisis temporal:

```python
df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d')
df.set_index('fecha', inplace=True)
```

## Resampling (Agrupación Temporal)

El resampling permite cambiar la frecuencia de los datos temporales:

```python
df.resample('M').sum()  # Agrupar por mes
df.resample('Q').mean()  # Agrupar por trimestre
df.resample('W').agg({'ventas': 'sum', 'precio': 'mean'})  # Múltiples agregaciones
```

## Rolling Windows (Ventanas Móviles)

Las ventanas móviles son esenciales para calcular promedios móviles y tendencias:

```python
df['ventas_ma7'] = df['ventas'].rolling(window=7).mean()  # Promedio móvil 7 días
df['ventas_std7'] = df['ventas'].rolling(window=7).std()  # Desviación estándar móvil
df['ventas_exp'] = df['ventas'].ewm(span=7).mean()  # Promedio móvil exponencial
```

## Shift y Diferencias Temporales

Análisis de cambios y tendencias:

```python
df['ventas_diff'] = df['ventas'].diff()  # Diferencia con el período anterior
df['ventas_shift'] = df['ventas'].shift(1)  # Desplazar valores
df['ventas_pct_change'] = df['ventas'].pct_change()  # Cambio porcentual
```

# Análisis de Correlaciones

![Análisis de Correlaciones](imgs/correlaciones.png)


El análisis de correlaciones es fundamental para entender las relaciones entre variables y seleccionar features para machine learning.

## Matriz de Correlación Básica

```python
df.corr()  # Matriz de correlación completa
df[['col1', 'col2']].corr()  # Correlación entre columnas específicas
```

## Métodos de Correlación

```python
df.corr(method='pearson')  # Correlación lineal (por defecto)
df.corr(method='spearman')  # Correlación de rangos
df.corr(method='kendall')  # Correlación de Kendall
```

## Identificación de Variables Altamente Correlacionadas

```python
# Encontrar pares de variables con alta correlación
corr_matrix = df.corr()
high_corr = (corr_matrix.abs() > 0.8) & (corr_matrix != 1.0)
correlated_vars = [(col, row) for col in high_corr.columns for row in high_corr.columns 
                  if high_corr.loc[row, col]]
```

# Tratamiento Avanzado de Valores Faltantes

![Tratamiento Avanzado de Valores Faltantes](imgs/valores_faltantes.png)


El manejo inteligente de valores faltantes es crucial para la calidad del análisis y modelos de machine learning.

## Detección Avanzada de Valores Faltantes

![Detección Avanzada de Valores Faltantes](imgs/valores_faltantes.png)


```python
df.isnull().sum()  # Conteo de nulos por columna
df.isnull().mean() * 100  # Porcentaje de valores faltantes
df.missing.ratio  # Ratio de valores faltantes (con missingno)
```

## Imputación por Grupos

```python
# Imputar con la media por grupo
df['ventas'].fillna(df.groupby('categoria')['ventas'].transform('mean'), inplace=True)

# Imputar con la mediana por grupo
df['precio'].fillna(df.groupby('region')['precio'].transform('median'), inplace=True)
```

## Interpolación Temporal

```python
df.interpolate(method='time')  # Interpolación basada en tiempo
df.interpolate(method='linear')  # Interpolación lineal
df.interpolate(method='polynomial', order=2)  # Interpolación polinómica
```

## Forward Fill y Backward Fill

```python
df.fillna(method='ffill')  # Forward fill
df.fillna(method='bfill')  # Backward fill
df.ffill(limit=3)  # Forward fill con límite
df.bfill(limit=2)  # Backward fill con límite
```

# Pivot Tables Complejas

![Pivot Tables Complejas](imgs/pivot_tables.png)


Las pivot tables son herramientas poderosas para análisis multidimensional y business intelligence.

## Pivot Table Básica

```python
df.pivot_table(index='categoria', columns='region', values='ventas', aggfunc='sum')
```

## Pivot Table con Múltiples Agregaciones

```python
df.pivot_table(
    index='categoria',
    columns='region', 
    values=['ventas', 'precio'],
    aggfunc={'ventas': ['sum', 'mean'], 'precio': ['mean', 'std']},
    fill_value=0
)
```

## Pivot Table con Totales

```python
df.pivot_table(
    index='categoria',
    columns='region',
    values='ventas',
    aggfunc='sum',
    margins=True,  # Agregar totales
    margins_name='Total'
)
```

## Funciones de Agregación Personalizadas

```python
def rango(series):
    return series.max() - series.min()

df.pivot_table(
    index='categoria',
    columns='region',
    values='ventas',
    aggfunc=rango
)
```

# Manejo de Texto y Datos Categóricos

![Manejo de Texto y Datos Categóricos](imgs/visualizacion.png)


El procesamiento de texto es fundamental para el análisis de datos no estructurados y la preparación de features.

## Operaciones Básicas de Texto

```python
df['texto'].str.lower()  # Convertir a minúsculas
df['texto'].str.upper()  # Convertir a mayúsculas
df['texto'].str.strip()  # Eliminar espacios en blanco
df['texto'].str.len()  # Longitud del texto
```

## Búsqueda y Reemplazo

```python
df['texto'].str.contains('palabra')  # Buscar patrón
df['texto'].str.replace('viejo', 'nuevo')  # Reemplazar texto
df['texto'].str.extract(r'(\d+)')  # Extraer números
df['texto'].str.split(' ')  # Dividir por separador
```

## Expresiones Regulares

```python
df['email'].str.extract(r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})')  # Extraer email
df['telefono'].str.extract(r'(\d{3})\D*(\d{3})\D*(\d{4})')  # Extraer teléfono
df['texto'].str.findall(r'\b[A-Z][a-z]+\b')  # Encontrar nombres propios
```

## Optimización de Datos Categóricos

```python
df['categoria'] = df['categoria'].astype('category')  # Convertir a categórico
df['categoria'].cat.categories  # Ver categorías
df['categoria'].cat.codes  # Ver códigos numéricos
```

# Performance y Optimización

![Performance y Optimización](imgs/performance.png)


La optimización es crucial para trabajar con datasets grandes y garantizar tiempos de ejecución eficientes.

## Operaciones Vectorizadas

```python
# ❌ Evitar loops
for i in range(len(df)):
    df.loc[i, 'nueva_col'] = df.loc[i, 'col1'] * df.loc[i, 'col2']

# ✅ Usar operaciones vectorizadas
df['nueva_col'] = df['col1'] * df['col2']
```

## Optimización de Memoria

```python
df.info(memory_usage='deep')  # Ver uso de memoria
df['col_int'] = df['col_int'].astype('int32')  # Reducir tamaño de dtype
df['col_str'] = df['col_str'].astype('category')  # Optimizar strings
```

## Chunking para Datasets Grandes

```python
# Procesar en chunks
chunk_list = []
chunk_size = 10000
for chunk in pd.read_csv('archivo_grande.csv', chunksize=chunk_size):
    chunk_processed = chunk.procesar_chunk()
    chunk_list.append(chunk_processed)
df_final = pd.concat(chunk_list, ignore_index=True)
```

## Profiling de Performance

![Profiling de Performance](imgs/performance.png)


```python
import time
start_time = time.time()
# Tu código aquí
end_time = time.time()
print(f"Tiempo de ejecución: {end_time - start_time:.4f} segundos")

# Usar %timeit en Jupyter
%timeit df.operacion()
```

# Joins y Merges Avanzados

![Joins y Merges Avanzados](imgs/joins.png)


La combinación correcta de datasets es fundamental para el análisis de datos integrados.

## Tipos de Join

```python
pd.merge(df1, df2, on='key', how='inner')  # Inner join
pd.merge(df1, df2, on='key', how='left')   # Left join
pd.merge(df1, df2, on='key', how='right')  # Right join
pd.merge(df1, df2, on='key', how='outer')  # Outer join
```

## Merge con Múltiples Claves

```python
pd.merge(df1, df2, on=['key1', 'key2'], how='inner')
pd.merge(df1, df2, left_on=['col1'], right_on=['col2'], how='left')
```

## Merge con Sufijos para Columnas Duplicadas

```python
pd.merge(df1, df2, on='key', suffixes=('_left', '_right'))
```

## Merge por Índice

```python
df1.merge(df2, left_index=True, right_index=True, how='left')
df1.join(df2, how='inner')  # Join por índice
```

# Detección y Tratamiento de Outliers

![Detección y Tratamiento de Outliers](imgs/outliers.png)


Los outliers pueden afectar significativamente los análisis y modelos. La detección y tratamiento apropiado es esencial.

## Detección con IQR (Rango Intercuartílico)

```python
Q1 = df['columna'].quantile(0.25)
Q3 = df['columna'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['columna'] < lower_bound) | (df['columna'] > upper_bound)]
```

## Detección con Z-Score

```python
from scipy import stats
z_scores = np.abs(stats.zscore(df['columna']))
outliers = df[z_scores > 3]
```

## Tratamiento de Outliers

![Tratamiento de Outliers](imgs/outliers.png)


```python
# Opción 1: Eliminar outliers
df_clean = df[(df['columna'] >= lower_bound) & (df['columna'] <= upper_bound)]

# Opción 2: Cap at bounds (Winsorizing)
df['columna_capped'] = df['columna'].clip(lower=lower_bound, upper=upper_bound)

# Opción 3: Reemplazar con mediana
median_value = df['columna'].median()
df.loc[outliers.index, 'columna'] = median_value
```

# Visualización con Pandas

![Visualización con Pandas](imgs/visualizacion.png)


Pandas incluye capacidades de visualización integradas que permiten exploración rápida de datos.

## Gráficos Básicos

```python
df['columna'].plot()  # Gráfico de línea
df['columna'].plot(kind='hist')  # Histograma
df.plot(x='col1', y='col2', kind='scatter')  # Scatter plot
df.plot(kind='box')  # Box plot
```

## Heatmap de Correlación

```python
import matplotlib.pyplot as plt
corr_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.show()
```

## Styling de DataFrames

```python
df.style.background_gradient()  # Gradiente de color
df.style.highlight_max()  # Resaltar máximos
df.style.format('{:.2f}')  # Formato de números
```

## Subplots Múltiples

```python
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
df['col1'].plot(ax=axes[0,0], kind='hist')
df['col2'].plot(ax=axes[0,1], kind='box')
df[['col1', 'col2']].plot(ax=axes[1,0])
df.plot.scatter(x='col1', y='col2', ax=axes[1,1])
plt.tight_layout()
```

# MultiIndex y Estructuras Complejas

![MultiIndex y Estructuras Complejas](imgs/multiindex.png)


Los índices múltiples permiten trabajar con datos jerárquicos y estructuras multidimensionales complejas.

## Creación de MultiIndex

![Creación de MultiIndex](imgs/multiindex.png)


```python
arrays = [['A', 'A', 'B', 'B'], ['uno', 'dos', 'uno', 'dos']]
index = pd.MultiIndex.from_arrays(arrays, names=['letra', 'numero'])
df = pd.DataFrame(np.random.randn(4, 2), index=index, columns=['col1', 'col2'])
```

## Acceso a MultiIndex

![Acceso a MultiIndex](imgs/multiindex.png)


```python
df.loc['A']  # Acceso por primer nivel
df.loc[('A', 'uno')]  # Acceso por ambos niveles
df.xs('uno', level='numero')  # Cross-section
```

## Pivot con MultiIndex

![Pivot con MultiIndex](imgs/multiindex.png)


```python
df.pivot_table(
    index='level1',
    columns=['level2', 'level3'],
    values='value',
    aggfunc='sum'
)
```

## Reset y Stack/Unstack

```python
df.reset_index()  # Resetear índice
df.stack()  # Convertir columnas a nivel de índice
df.unstack()  # Convertir nivel de índice a columnas
```